In [1]:
import os
import glob
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
"""
Importante: Os diretórios, in e out são baseados no mapeamento da máquina onde estão os dados!
Caso queira adaptar, é importante verificar a possibilidade de mudar essas variáveis e o código abaixo
de mudança de pasta. 
"""
os.chdir('../..') #Retorna duas bastas.

In [3]:
path_in = os.path.join(os.getcwd(),r'DADOS\SHP\METRICAS SIMPLES')

In [ ]:
path_out = os.path.join(os.getcwd(),r'DADOS\TESTE')

In [5]:
files = glob.glob(path_in + '\*.shp')

In [27]:
for file in files:
    #Pegar o nome do arquivo
    nm_arquivo = os.path.basename(file).split('.')[0]

    #Ler o shp e transformar em um geodf
    gdf = gpd.read_file(file)
    
    #Transformar em um dataframe e dropar a coluna geometry
    df = pd.DataFrame(gdf.drop(columns='geometry'))

    #Agrupar as colunas DN, nome e legenda e resetar o index
    df2 = df.groupby(['DN','nome','legenda']).sum(['area_m2','perimetro']).reset_index()

    #Ordenar pela Área de Proteção Ambiental
    df2 = df2.sort_values(by=['nome'])

    #Criar a coluna Classificação
    df2['Classificação'] = df2['DN'].astype(str) + ' - ' + df2['legenda'].astype(str)

    #Ordenar as colunas
    df2 = df2[[ 
                'nome',
                'Classificação',
                'area_m2',
                'perimetro'
            ]]
    
    #Renomear as colunas
    df2 = df2.rename({
                        'nome':'Nome',
                        'area_m2':'Área (m²)',
                        'perimetro':'Perímetro'
                },
                axis='columns'
                )

    #Salvar em csv
    df2.to_csv(path_out + f'\{nm_arquivo}.csv',
               encoding='latin',
               index=False)
    